### Model to reassign all true uniques: set coverage

Read individual data and convert

- Geographic level: tract
- Attributes: VOTINGAGE (2) $*$ HISPANIC (2) $*$ CENRACE (63)

In [7]:
import pandas as pd
import numpy as np

filename_hist = 'data/franklin_hist.csv'
hist = pd.read_csv(filename_hist)

# block to tract
hist['TRACT'] = hist['GEOID10'].astype(str).str[:11]
col_names = hist.columns.to_numpy()
col_names = np.delete(col_names, [0, -1])
hist = hist.groupby('TRACT').sum()[col_names]
hist = hist.reset_index()
hist

,TRACT,00000000,00000001,00000002,00000003,00000004,00000005,00000006,00000007,00000008,...,07010153,07010154,07010155,07010156,07010157,07010158,07010159,07010160,07010161,07010162
0,39049000110,438,6,0,7,0,4,5,1,0,...,0,0,0,0,0,0,0,0,0,0
1,39049000120,467,4,0,11,0,2,2,0,2,...,0,0,0,0,0,0,0,0,0,0
2,39049000210,430,11,0,6,0,4,4,1,4,...,0,0,0,0,0,0,0,0,0,0
3,39049000220,653,3,1,12,0,6,7,0,6,...,0,0,0,0,0,0,0,0,0,0
4,39049000310,313,205,0,17,0,2,19,0,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,39049010500,1821,35,1,409,0,2,5,2,22,...,0,0,0,0,0,0,0,0,0,0
280,39049010601,1408,36,0,146,0,3,12,0,15,...,0,0,0,0,0,0,0,0,0,0
281,39049010602,1585,45,3,139,0,4,7,2,9,...,0,0,0,0,0,0,0,0,0,0
282,39049010700,83,11,0,4,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# HHGQ (8) $*$ VOTINGAGE (2) $*$ HISPANIC (2) $*$ CENRACE (63) to VOTINGAGE (2) $*$ HISPANIC (2) $*$ RACE (7)
n2, n3, n4 = 2, 2, 63

for y in range(n2):  # voting age
    y = '{number:0{width}d}'.format(width=2, number=y)
    col_names = [col for col in hist.columns if y in col[2:4] and len(col)==8]

    for z in range(n3):  # ethnicity
        z = '{number:0{width}d}'.format(width=2, number=z)
        col_names2 = [col for col in col_names if z in col[4:6]]

        col_two_or_more_races = []
        for x in range(n4):  # race
            if x >= 0 and x <= 5:
                x = '{number:0{width}d}'.format(width=2, number=x)
                col_names3 = [col for col in col_names2 if x in col[6:8]]
                hist[x + y + z] = hist[col_names3].sum(axis=1)
            else:
                x = '{number:0{width}d}'.format(width=2, number=x)
                col_names3 = [col for col in col_names2 if x in col[6:8]]
                col_two_or_more_races.extend(col_names3)
        hist['06' + y + z] = hist[col_two_or_more_races].sum(axis=1)

hist.drop([col for col in hist.columns if len(col)==8], axis=1, inplace=True)
hist            

,TRACT,000000,010000,020000,030000,040000,050000,060000,000001,010001,...,040100,050100,060100,000101,010101,020101,030101,040101,050101,060101
0,39049000110,438,6,0,7,0,4,6,29,0,...,0,2,1,17,3,0,0,0,6,0
1,39049000120,467,4,0,11,0,2,6,13,0,...,1,0,3,18,0,0,0,0,4,0
2,39049000210,430,11,0,6,0,4,13,10,0,...,0,2,4,34,0,0,0,0,4,0
3,39049000220,653,3,1,12,0,6,13,20,0,...,0,6,4,16,0,2,0,0,8,1
4,39049000310,313,205,0,17,0,2,25,15,1,...,2,5,8,40,1,1,0,0,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,39049010500,1821,35,1,409,0,2,33,27,0,...,1,8,8,70,0,1,0,0,19,0
280,39049010601,1408,36,0,146,0,3,32,16,3,...,0,2,4,86,0,0,0,0,13,1
281,39049010602,1585,45,3,139,0,4,19,13,1,...,0,8,11,71,2,2,0,0,25,0
282,39049010700,83,11,0,4,0,0,2,6,1,...,1,2,2,10,0,0,0,0,14,0


Disclosure risks before protection

In [20]:
import numpy as np

pd.set_option('use_inf_as_na', True)
a = hist.iloc[:,1:].apply(lambda x: 1 / x).to_numpy()
a[~np.isfinite(a)] = 0
print("Identification prob: ", np.sum(a) / a.size)

Identification prob:  0.1667384238525854


In [21]:
import numpy as np

u = hist.iloc[:,1:]
print("Unique prob: ", np.count_nonzero(u == 1) / u.size)

Unique prob:  0.08601609657947686


Model inputs

In [16]:
import numpy as np

# define all the input data for the model
I, K = hist.shape[0], hist.shape[1] - 1
nj = 20
r = 3

V = []
for k in range(1, K+1):
    V.append(hist.index[(hist.iloc[:,k] <= r) & (hist.iloc[:,k] > 0)].tolist())

count = 0
for listElem in V:
    count += len(listElem)  
print(count)

A = hist.iloc[:,1:].to_numpy()
print(A.shape, A[0])

W = hist.iloc[:,1:].apply(lambda x: 1 + 1 / x).to_numpy()
W = np.nan_to_num(W, posinf=10) 
print(W.shape, W[0])

1497
(284, 28) [ 438    6    0    7    0    4    6   29    0    1    0    0    1    1
 2733   31    1   29    0    2    1   17    3    0    0    0    6    0]
(284, 28) [ 1.00228311  1.16666667 10.          1.14285714 10.          1.25
  1.16666667  1.03448276 10.          2.         10.         10.
  2.          2.          1.0003659   1.03225806  2.          1.03448276
 10.          1.5         2.          1.05882353  1.33333333 10.
 10.         10.          1.16666667 10.        ]


Coverage I: all except the origin and other uniques

In [17]:
import numpy as np

## define coverage aijk
T = np.ones((I, I, K))

for i in range(I): 
    for j in range(I):
        for k in range(K):
            if i == j or j in V[k]:
                T[i, j, k] = 0
T   

array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1.

Coverage II: neighboring

In [27]:
import numpy as np
import geopandas as gpd
from pysal.lib import weights

filename_gdf = 'data/franklin_tract10.json'
gdf = gpd.read_file(filename_gdf)
gdf['GEOID10'] = gdf['GEOID10'].astype(str)
wr = weights.distance.KNN.from_dataframe(gdf, k=10)
print(wr.neighbors[0])

## define coverage aijk
T = np.zeros((I, I, K))
for i in wr.neighbors:
    neighbors_idx = wr.neighbors[i]
    for j in neighbors_idx:
        geoid = gdf.loc[[j],'GEOID10'].values[0]
        idx = hist.loc[hist["TRACT"] == geoid].index[0]
        for k in range(K):
            if j not in V[k]:
                T[i, j, k] = 1
T        

[4, 90, 89, 276, 12, 123, 171, 1, 180, 13]


array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

Run model

In [18]:
import pandas as pd
from gurobipy import Model, GRB, LinExpr, quicksum

# initialize model
m = Model('td')
# m.Params.LogToConsole = 0

# add objective function
obj = LinExpr()

# add decision variables and objective function
theta = {}
for k in range(K):
    if len(V[k]) == 0:
        continue
    for i in V[k]:
        for j in range(I):
            # decision variables
            theta[i, j, k] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name="theta_%d_%d_%d"%(i, j, k))
            # m.update()
            # objective
            obj += theta[i, j, k] * A[i, k] * W[j, k]


m.setObjective(obj, GRB.MINIMIZE)

# add constraints
for k in range(K):
    if len(V[k]) == 0:
        continue
    for i in V[k]:
        m.addConstr(quicksum(theta[i, j, k] for j in range(I)) == 1)
        m.addConstr(quicksum(T[i, j, k] * theta[i, j, k] for j in range(I)) == 1)

for j in range(I):
    m.addConstr(quicksum(quicksum(theta[i, j, k] * A[i, k] for i in V[k]) for k in range(K)) <= nj)

m.update()
m.optimize()

Academic license - for non-commercial use only - expires 2022-08-05
Using license file C:\Users\10716\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3278 rows, 425148 columns and 1145003 nonzeros
Model fingerprint: 0xe6f7c8ca
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 1.50s
Presolved: 3278 rows, 425148 columns, 1145003 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.876e+04
 Factor NZ  : 6.437e+04 (roughly 10 MBytes of memory)
 Factor Ops : 1.140e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.51534171e

In [ ]:
for var in m.getVars():
    print(var.VarName, var.X)

Disclosure risks after protection

In [50]:
theta_all = np.empty([I, I, K])
for k in range(K):
    for i in range(I):
        if i not in V[k] and A[i, k] != 0:
            theta_all[i, i, k] = 1

for var in m.getVars():
    name = var.VarName.split("_")
    theta_all[int(name[1]), int(name[2]), int(name[3])] = var.X

# check
np.sum(theta_all, axis=1)

array([[1., 1., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
p = np.ones([I, K])
b = hist.iloc[:,1:].to_numpy()
for k in range(K):
    for i in range(I):
        sum = 0
        for j in range(I):
            if i != j:
                sum += theta_all[j, i, k] * b[j, k]
        p[i, k] = theta_all[i, i, k] / (theta_all[i, i, k] * b[i, k] + sum)
p[~np.isfinite(p)] = 0
print("Identification prob: ", np.sum(p) / p.size)

C:\Users\10716\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


Identification prob:  0.035466224755920275


In [52]:
v = 0
for k in range(K):
    for i in range(I):
        if b[i, k] == 1 and theta_all[i, i, k] == 1:
            v += 1

print("Unique prob: ", np.sum(v) / (I * K))

Unique prob:  0.0
